In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
# new_version_path = Path(r'D:\vnpy-2.0.8')
new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

from datetime import datetime
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from vnpy.trader.constant import Interval
from vnpy.trader.database import database_manager
from vnpy.trader.utility import extract_vt_symbol

from double_ma_strategy import DoubleMaStrategy
from utility import vt_bar_to_df, vt_trade_to_df


def load_data(vt_symbol: str, interval: str, start: datetime, end: datetime) -> pd.DataFrame:
    symbol, exchange = extract_vt_symbol(vt_symbol)
    data = database_manager.load_bar_data(
        symbol,
        exchange,
        Interval(interval),
        start=start,
        end=end,
    )

    return vt_bar_to_df(data)

In [ ]:
# 从数据库载入数据
price = load_data('RB888.SHFE', '1h', datetime(2019, 1, 1), datetime(2019, 12, 1))
print(f"记录数：{len(price)} 重复数：{sum(price.index.duplicated())}")
price.head()

In [ ]:
# 用matplot的交互式窗口查看数据是否正常
# 按住右键上下左右移动可以上下左右缩放，按住左键可以平移
price2 = price.reset_index(inplace=True)
price2.head()

%matplotlib notebook
axe = price2.close.plot(figsize=(12, 6))
axe.scatter([2, 10], [2596, 2612], color='b')
plt.show()

In [ ]:
# 设置回测参数

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="RB888.SHFE",
    interval="1h",
    start=datetime(2019, 1, 1),
    end=datetime(2019, 12, 1),
    rate=1/10000,
    slippage=1,
    size=10,
    pricetick=1,
    capital=100000,
)
engine.add_strategy(DoubleMaStrategy, {})

In [ ]:
# 读取数据
engine.load_data()
engine.run_backtesting()
# df = engine.calculate_result()
# df.to_csv('result.csv')
# engine.calculate_statistics()
# engine.show_chart()

In [ ]:
trades = engine.get_all_trades()
trade_df = vt_trade_to_df(trades)
trade_df.head()

In [ ]:
t2 = trade_df.drop_duplicates()
t2.head()
price.loc[t2.index]

In [ ]:
# 进行连续合约的回测

In [ ]:
# 进行分段合约的回测